# 6. Short-Term Memory

Must read:
- [What is Memory?](https://langchain-ai.github.io/langgraph/concepts/memory/#what-is-memory)

In [1]:
from langchain_ollama import ChatOllama
import os
from colorama import Fore, Style

In [100]:
# Initialize the language model with specific settings
llm = ChatOllama(
    base_url=os.getenv("OLLAMA_SERVER"),
    model = "llama3.2:latest",
    temperature = 0.4
)

In [101]:
# Create a verys simple short-term memory to store conversation history
class ShortTermMemory:
    def __init__(self, max_tokens=1024):
        self.history = []  # To store conversation history
        self.max_tokens = max_tokens  # Maximum allowed token count

    def add_message(self, role, content):
        self.history.append({"role": role, "content": content})

    def get_trimmed_history(self):
        total_tokens = sum(len(msg["content"].split()) for msg in self.history)
        while total_tokens > self.max_tokens and self.history:
            total_tokens -= len(self.history.pop(0)["content"].split())
        return self.history

    def reset(self):
        self.history = []

In [102]:
# Initialize short-term memory
memory = ShortTermMemory(max_tokens=1024)
# Reset the memory
memory.reset()

In [103]:
# Define the chat function
# it will take user input and return the AI response
# then it will store the conversation history in the memory
def chat(user_input):
    system_message = {"role": "system", "content": "Please respond to all messages in French."}
    memory.add_message("user", user_input)
    prompt = [system_message] + memory.get_trimmed_history()
    response = llm.invoke(prompt)
    ai_reply = response.content
    ai_reply = "\n".join(line for line in ai_reply.splitlines() if line.strip())
    memory.add_message("assistant", ai_reply)    
    return ai_reply

In [107]:
# Some system initialization messages
memory.add_message("system", "Always answer in French with short and precise sentences.")
# A list of questions to challenge them memory
test_inputs = [
    "My name is Alex?",
    "What's the capital of France?",
    "Who won the World Cup in 2018?",
    "Do you remember my name?",
    "Can you tell me what we talked about earlier?",
    "What is the tallest mountain in the world?",
    "Give me all questions I asked you so far",
    "What did we discuss about my name?",
    "What is 2+2?",
    "What did I ask you about the World Cup?",
]

In [ ]:
# Start the conversation
# You can put this in a While True loop to keep the conversation going
# But it's not nice in a Jupyter notebook
for i, user_input in enumerate(test_inputs, start=1):
    print("-" * 100)
    print(Fore.GREEN +f"You: {i}. {user_input}")
    reply = chat(user_input)
    print(Fore.YELLOW +f"IA: {reply}")
print(Style.RESET_ALL)

In [ ]:
# What's in the memory?
mem = memory.get_trimmed_history()
for m in mem:
    print(m["role"], m["content"])